In [1]:
%load_ext lab_black
# Threading used for concurrency
# Mostly used to handle io bound app, means there are a lot of waiting operation

import time
import threading

# Normal Function Without Thread
def do_something(wait):
    print("begin something")
    time.sleep(wait)
    print("something finished")


print("\n\n-- normal function --\n\n")
start = time.perf_counter()
do_something(1)
do_something(1)
finish = time.perf_counter()
print(f"{round(finish - start, 2)} secods elapsed")



-- normal function --


begin something
something finished
begin something
something finished
2.0 secods elapsed


In [2]:
print("\n\n-- with thread function --\n\n")
start = time.perf_counter()

thread_1 = threading.Thread(target=do_something, args=[1])
thread_2 = threading.Thread(target=do_something, args=[1])

# execute function on separate thread
thread_1.start()
thread_2.start()

# join master thread, or current thread
thread_1.join()
thread_2.join()

finish = time.perf_counter()
print(f"{round(finish - start, 2)} secods elapsed")



-- with thread function --


begin something
begin something
something finished
something finished
1.0 secods elapsed


In [3]:
print("\n\n-- with thread list function --\n\n")
start = time.perf_counter()

threads = []

for _ in range(10):
    thread = threading.Thread(target=do_something, args=[1])
    thread.start()
    threads.append(thread)

for current_thread in threads:
    current_thread.join()

finish = time.perf_counter()
print(f"{round(finish - start, 2)} secods elapsed")



-- with thread list function --


begin somethingbegin something

begin something
begin something
begin something
begin something
begin something
begin somethingbegin something

begin something
something finished
something finished
something finished
something finished
something finished
something finished
something finished
something finished
something finished
something finished
1.02 secods elapsed


In [4]:
# Thread with ThreadPoolExecutor & submit
from concurrent import futures


def do_something_with_return(wait):
    print("begin something")
    time.sleep(wait)
    return f"something finished in {wait}"


print("\n\n-- with thread list function --\n\n")
start = time.perf_counter()

with futures.ThreadPoolExecutor() as executor:
    args = [1, 2, 3, 4]
    result = [executor.submit(do_something_with_return, arg) for arg in args]

    for f in futures.as_completed(result):
        print(f.result())

finish = time.perf_counter()
print(f"{round(finish - start, 2)} secods elapsed")



-- with thread list function --


begin something
begin something
begin something
begin something
something finished in 1
something finished in 2
something finished in 3
something finished in 4
4.01 secods elapsed


In [5]:
# Thread with ThreadPoolExecutor & map
# Use this if u want the result in the same order

print("\n\n-- with futures thread function --\n\n")
start = time.perf_counter()
with futures.ThreadPoolExecutor() as executor:
    args = [1, 2, 3, 4]
    results = executor.map(do_something_with_return, args)

#     for result in results:
#         print(result)

finish = time.perf_counter()
print(f"{round(finish - start, 2)} secods elapsed")



-- with futures thread function --


begin something
begin something
begin something
begin something
4.01 secods elapsed


In [6]:
# Real World Example

import requests

img_urls = [
    "https://c.ndtvimg.com/2020-03/og642c2_ferrari-f8-tributo-review_625x300_12_March_20.jpg",
    "https://cdn2.iconfinder.com/data/icons/iconslandtransport/PNG/256x256/CarGrey.png",
    "https://www.emoji.co.uk/files/apple-emojis/travel-places-ios/483-racing-car.png",
]


def download_image(url):
    img_bytes = requests.get(url).content
    image_name = url.split("/")[3]
    image_name = f"download/{image_name}.jpg"

    with open(image_name, "wb") as file:
        file.write(img_bytes)
        print("Finish Download Image : {}".format(image_name))


print("Download without thread")
start = time.perf_counter()

for img_url in img_urls:
    download_image(img_url)

finish = time.perf_counter()
print(f"{round(finish-start,2)} second elapsed")

Download without thread
Finish Download Image : download/2020-03.jpg
Finish Download Image : download/data.jpg
Finish Download Image : download/files.jpg
5.66 second elapsed


In [7]:
print("Download with thread")
start = time.perf_counter()

with futures.ThreadPoolExecutor() as executor:
    executor.map(download_image, img_urls)

finish = time.perf_counter()
print(f"{round(finish-start,2)} second elapsed")

Download with thread
Finish Download Image : download/data.jpg
Finish Download Image : download/2020-03.jpg
Finish Download Image : download/files.jpg
1.3 second elapsed
